In [234]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os
import pickle

In [235]:
# 1. Chargement des données
cols_to_load = [
    'TUACTDUR24',    # Durée de l'activité
    'ACTIVITY_NAME', # Nom de l'activité
    'TUSTARTTIM',    # Heure de début (format HH:MM:SS)
    'TUDIARYDAY'     # Jour de la semaine
]

In [237]:
df = pd.read_csv("../Data/raw/atus_full_selected.csv", usecols=cols_to_load)

In [238]:
# 2. Nettoyage des données
df = df.dropna()
df = df.reset_index(drop=True)

In [239]:
# 3. Nouvelle fonction de conversion du temps
def time_to_minutes(t):
    """Convertit HH:MM:SS en minutes depuis minuit"""
    try:
        hh, mm, ss = map(int, t.split(':'))
        return hh * 60 + mm
    except:
        return 0  # Valeur par défaut si conversion échoue


In [240]:
# Application de la conversion
df['START_MINUTES'] = df['TUSTARTTIM'].apply(time_to_minutes)


In [241]:
# 4. Vérification des conversions
print("Exemples de conversion:")
print(df[['TUSTARTTIM', 'START_MINUTES']].head())


Exemples de conversion:
  TUSTARTTIM  START_MINUTES
0   04:00:00            240
1   05:00:00            300
2   05:30:00            330
3   15:30:00            930
4   18:00:00           1080


In [242]:
# 5. Encodage et autres transformations
activity_encoder = LabelEncoder()
df['ACTIVITY_CODE'] = activity_encoder.fit_transform(df['ACTIVITY_NAME'])
df['DAY_OF_WEEK'] = df['TUDIARYDAY'] - 1  # 0=dimanche, 6=samedi
df['IS_WEEKEND'] = (df['DAY_OF_WEEK'] >= 5).astype(int)


In [243]:
# 6. Sélection finale des colonnes
final_columns = [
    'ACTIVITY_CODE',
    'TUACTDUR24',
    'START_MINUTES',
    'DAY_OF_WEEK',
    'IS_WEEKEND'
]
final_df = df[final_columns]

In [244]:
# 7. Sauvegarde
os.makedirs("../Data/processed", exist_ok=True)
final_df.to_csv("../Data/processed/cleaned_data.csv", index=False)

In [245]:
with open("../Data/processed/activity_encoder.pkl", "wb") as f:
    pickle.dump(activity_encoder, f)